# Setup

In [1]:
import Utils.model_file as model_file
import Utils.utils_file as utils_file
import Utils.scraping_file as scraping_file
from Utils.constants import *

import numpy as np
import pandas as pd
import sklearn.metrics as skm

In [2]:
Model = model_file.Final_Model_Class(target='Score', model_type='logistic')
Scraping = scraping_file.Scraping_Class()
Utils = Model.Utils
Utils.TARGET_DF = Utils.TARGET_DF[Utils.TARGET_DF[Utils.TARGET]!=2]

In [3]:
validation_df = Utils.TARGET_DF.copy()
validation_df = validation_df[validation_df['realSemesterYear'] == CURRENT_YEAR_SEMESTER]
print(len(validation_df))

500


In [4]:
print(np.mean(Utils.regions_stats['error']))
Utils.regions_stats.head()

0.17136666666666667


,region,model,test_size,train_size,error,cut_off_var,threshold,len_lost
0,LFL,0,478,2675,0.292,1.5,0.35,0.62
1,LPL,10,801,6885,0.188,1.5,0.35,0.64
2,Prime_Tier2,0,128,3315,0.191,1.5,0.35,0.73
3,Ultraliga,0,234,9785,0.129,1.5,0.35,0.71
4,SuperLiga_Tier2,2,120,4039,0.000,1.5,0.35,1.00


# Auto validation

In [5]:
final_df = pd.DataFrame()
regions_stats_proof = pd.DataFrame(columns=['region','error','len lost','len', 'true_len'])
Utils.logistic_threshold = 0.35
final_len = 0

for tournament in validation_df.tournamentId.unique():
    print('============')
    print(f'Tournament: {tournament}')

    region_abrev = tournament[:-5]
    if region_abrev not in Utils.regions_feature_cols:
        continue
    # if region_abrev == 'LCK':
    #     continue

    old_error = Utils.regions_stats[Utils.regions_stats['region']==region_abrev].error.iloc[0]
    target_df = Utils.TARGET_DF.copy()
    feature_columns = Utils.regions_feature_cols[region_abrev]
    train_data = Utils.regions_train_data[region_abrev]
    target_columns = [x for x in list(target_df.columns) if x.replace('Team_Red_','').replace('Team_Blue_','') in feature_columns]

    train_df = target_df[target_df['regionAbrev'].isin(train_data)].copy()
    train_df = train_df[train_df['realSemesterYear'].astype(int) < int(CURRENT_YEAR_SEMESTER)]
    train_df = train_df[target_columns + Utils.INFO_COLS]
    xtrain = train_df.drop(['Date', Utils.TARGET] + OFF_COLS, axis=1, errors='ignore').copy()
    ytrain = train_df[Utils.TARGET]

    prediction_df = validation_df[validation_df['tournamentId'] == tournament].copy()
    prediction_df = prediction_df[target_columns + Utils.INFO_COLS]
    xtest = prediction_df.drop(['Date', Utils.TARGET] + OFF_COLS, axis=1, errors='ignore').copy()
    ytest = prediction_df[Utils.TARGET]
    
    model_number = Utils.regions_stats[Utils.regions_stats.region == region_abrev].model.iloc[0]

    metric, prediction = Utils.make_pred(model_number=model_number, xtrain=xtrain, ytrain=ytrain, xtest=xtest, ytest=ytest, reps=1)
    true_len = round(len(ytest)*abs(Utils.len_lost-1))

    df_to_concat = pd.DataFrame([prediction, ytest]).transpose()
    df_to_concat['region'] = region_abrev
    final_df = pd.concat([final_df,df_to_concat])

    df_to_concat = pd.DataFrame([region_abrev, metric, Utils.len_lost, len(ytest), true_len]).transpose()
    df_to_concat.columns = ['region','error','len lost','len','true_len']
    regions_stats_proof = pd.concat([regions_stats_proof,df_to_concat])

    print(f'\nExpected error: {old_error}')
    print(f'Real error: {metric}')
    print(f'Len: {len(ytest)}')
    print(f'Len lost: {Utils.len_lost}')
    print(f'True len: {true_len}')


final_df_filter = final_df[(final_df[0]<=Utils.logistic_threshold) | (final_df[0]>=1-Utils.logistic_threshold)]
final_df_filter[0] = final_df_filter[0].round()
final_df_filter['error'] = final_df_filter[0]!=final_df_filter[1]
mean_real_error = skm.mean_absolute_error(round(final_df_filter[0]), final_df_filter[1])
all_len_lost = len(final_df) - len(final_df_filter)
regions_stats_proof = regions_stats_proof[regions_stats_proof['true_len']>0]

print('\n===============\n')
print(f'Mean real error: {mean_real_error}')
print(f'Len lost: {all_len_lost}')
print(f'Start len: {len(final_df)}')
print(f'Final len: {len(final_df_filter)}')


Tournament: Hitpoint_Tier220231

Expected error: 0.094
Real error: 0.0
Len: 12
Len lost: 0.83
True len: 2
Tournament: LFL20231

Expected error: 0.292
Real error: 0.182
Len: 40
Len lost: 0.72
True len: 11
Tournament: LLA20231

Expected error: 0.211
Real error: 0.25
Len: 18
Len lost: 0.56
True len: 8
Tournament: LPL20231

Expected error: 0.188
Real error: 0.217
Len: 97
Len lost: 0.76
True len: 23
Tournament: LCO20231
no data left after filtering

Expected error: 0.12
Real error: 1.0
Len: 10
Len lost: 1.0
True len: 0
Tournament: LCK_Tier220231

Expected error: 0.283
Real error: 0.6
Len: 39
Len lost: 0.62
True len: 15
Tournament: NLC20231

Expected error: 0.184
Real error: 0.667
Len: 16
Len lost: 0.44
True len: 9
Tournament: GLL20231

Expected error: 0.083
Real error: 0.182
Len: 20
Len lost: 0.45
True len: 11
Tournament: Elite_Tier220231

Expected error: 0.254
Real error: 0.2
Len: 12
Len lost: 0.58
True len: 5
Tournament: Prime_Tier220231

Expected error: 0.191
Real error: 0.083
Len: 26
Le

In [5]:
regions_stats_proof.sort_values(by='error')

NameError: name 'regions_stats_proof' is not defined

# Manual validation

In [6]:
Model.print_region_teams('LPLOL')

{0: 'ASP Esports', 1: 'Boavista FC', 2: 'Byteway Esports', 3: 'Cidade Curiosa', 4: 'Doxa Gaming', 5: 'EFIVE Esports', 6: 'EGN Esports', 7: 'Electronik Generation', 8: 'Estoril Praia eSports', 9: 'For The Win', 10: 'For The Win Esports', 11: 'For The Win eSports Club', 12: 'GTZ Bulls', 13: 'GTZ Esports', 14: 'GeekCase eSports', 15: 'Grow Up eSports', 16: 'Hexagone eSports', 17: 'K1CK eSports Club', 18: 'Karma Clan Esports', 19: 'Keypulse Esports', 20: 'OFFSET Esports', 21: 'Odivelas Sports Club', 22: 'SAMCLAN Esports', 23: 'SAMCLAN Esports Club', 24: 'White Dragons'}


In [7]:
Model.blue_team = 19
Model.red_team = 6
Model.print_team_players()

#team_blue_list=['Wayward', 'Tian', 'RooKie', 'JackeyLove', 'Mark']
#team_red_list=['Zika', 'Tarzan', 'Scout', 'GALA', 'Hang']

team_blue_list = ['Jackie', 'RapMonsters', 'Qu4rtzo', 'Lyrokun', 'sNaN']
team_red_list = ['Vortum', 'Lulas', 'Cayetano', 'zekja', 'Tochas']


In [10]:
team_blue_list = ['HUGOD', 'RapMonsters', 'Qu4rtzo', 'Lyrokun', 'Shieda']
team_red_list = ['Vortum', 'Lulas', 'Cayetano', 'zekja', 'Tochas']

In [11]:
Model.make_prediction(manual_insert=True, team_blue_list=team_blue_list, team_red_list=team_red_list)

last train data: 2023-06-15 00:00:00
no data names on: ['zekja']
0.8865574994520661


In [ ]:
Utils.bet_ratio_vars(result=0.1, ratio=1.6, chance=None)